# three_level_segmentation - Uses pyclesperanto kernel

### Import Modules

In [1]:
import os
import time
from multiprocessing import Pool

import numpy as np
from tifffile import imread, imsave

from skimage.filters import threshold_otsu
from skimage.morphology import dilation, erosion, remove_small_objects
from skimage.measure import label, regionprops, marching_cubes
from skimage.feature import peak_local_max
from skimage import img_as_float

from scipy.ndimage import gaussian_filter, binary_fill_holes
from scipy import ndimage, signal

import matplotlib.pyplot as plt

if 'BINDER_SERVICE_HOST' in os.environ:
    os.environ['DISPLAY'] = ':1.0'
    
import napari

## Pad the image with its median value.

In [2]:
def add_median_border(image_data):
    (z_len, y_len, x_len) = image_data.shape
    median_intensity = np.median(image_data)
    padded_image_data = np.full((z_len+2, y_len+2, x_len+2), median_intensity)
    padded_image_data[1:z_len+1, 1:y_len+1, 1:x_len+1]=image_data
    return padded_image_data

### Make a 3D Spherical Structured Element 

In [3]:
def make_sphere_3D(radius):
    radius = int(radius)
    sphere = np.zeros((radius, radius, radius))
    (z_len, y_len, x_len) = sphere.shape
    for i in range(int(z_len)):
        for j in range(int(y_len)):
            for k in range(int(x_len)):
                if ((i**2+j**2+k**2)/radius**2) < 1:
                    sphere[i,j,k]=1
    return sphere

### Create an "inside" image
Original algorithm fills holes plane by plane in z.
https://github.com/DanuserLab/u-shape3D/blob/a6dcfbcce58550264dfc330c8bfb7b4720e36893/Mesh/threeLevelSegmentation3D.m

In [4]:
def make_inside_image(padded_image_data, insideGamma,insideBlur, insideDilateRadius, insideErodeRadius):
    image_blurred = padded_image_data**insideGamma
    image_blurred = gaussian_filter(image_blurred, sigma=insideBlur)
    image_threshold = threshold_otsu(image_blurred)

    image_binary = image_blurred > image_threshold                   
    image_binary = dilation(image_binary, make_sphere_3D(insideDilateRadius))
    image_binary = ndimage.binary_fill_holes(image_binary)
    image_binary = np.double(erosion(image_binary, make_sphere_3D(insideErodeRadius)))
    inside_image = gaussian_filter(image_binary, sigma=1)
    return inside_image



### Create a Normalized Cell Image

In [5]:
def make_normalized_image(input):
    image_threshold = threshold_otsu(padded_image_data)
    normalized_cell = padded_image_data - image_threshold
    normalized_cell = normalized_cell/np.std(normalized_cell)
    return normalized_cell



### Create a Surface Cell Image
https://github.com/DanuserLab/u-shape3D/blob/a6dcfbcce58550264dfc330c8bfb7b4720e36893/Mesh/surfaceFilterGauss3D.m

Convolution Sequence:

d2z -> [d2g, g, g]

d2y -> [g, d2g, g]

d2x -> [g, g, d2g]


In [6]:
def surface_filter_gauss_3D(input, sigma):
    
    # Same Sigma Value for All 3 Dimensions
    w = np.ceil(5*sigma)
    x = np.arange(-w, w, 1)
    g = np.zeros(x.shape)

    # Calculate 1D Gaussian
    for i in range(int(x.size)):
        g[i] = np.exp(-x[i]**2/(2*sigma**2))

    # Calculate Second Derivative of 1D Gaussian
    d2g = np.zeros(x.shape)
    for i in range(int(x.size)):
        d2g[i] = (-(x[i]**2/sigma**2 - 1) / sigma**2)*(np.exp(-x[i]**2/(2*sigma**2)))

    gSum = np.sum(g);

    # Gaussian Kernels
    g = g/gSum; #1D Gaussian
    d2g = d2g/gSum; #1D Second Derivative Kernel

    d2z_image = signal.fftconvolve(input, d2g[:, None, None], mode='same')
    d2z_image = signal.fftconvolve(d2z_image, g[None,: , None], mode='same')
    d2z_image = signal.fftconvolve(d2z_image, g[None, None, :], mode='same')

    d2y_image = signal.fftconvolve(input, d2g[None, :, None], mode='same')
    d2y_image = signal.fftconvolve(d2y_image, g[None, None,:], mode='same')
    d2y_image = signal.fftconvolve(d2y_image, g[:, None, None], mode='same')

    d2x_image = signal.fftconvolve(input, d2g[None, None, :], mode='same')
    d2x_image = signal.fftconvolve(d2x_image, g[:, None, None], mode='same')
    d2x_image = signal.fftconvolve(d2x_image, g[None, :, None], mode='same')
        
    
    return d2z_image, d2y_image, d2x_image

### Multiscale Surface Filter 
https://github.com/DanuserLab/u-shape3D/blob/a6dcfbcce58550264dfc330c8bfb7b4720e36893/Mesh/multiscaleSurfaceFilter3D.m

In [7]:
def multiscale_surface_filter_3D(input, scales):
    n_scales = np.size(scales)
    max_response = np.zeros(np.shape(input))
    max_response_scale = np.zeros(np.shape(input))

    for i in range(n_scales):
        d2z_temp, d2y_temp, d2x_temp = surface_filter_gauss_3D(input,scales[i])
        d2z_temp[d2z_temp<0] = 0
        d2y_temp[d2y_temp<0] = 0
        d2x_temp[d2x_temp<0] = 0

        sMag = np.sqrt(d2z_temp**2 + d2y_temp**2 + d2x_temp**2)
        is_better = sMag > max_response
        max_response_scale[is_better] = i
        max_response[is_better] = sMag[is_better]
    
    surface_background_mean = np.mean(max_response)
    surface_background_std = np.std(max_response)
    surface_threshold = surface_background_mean + (nSTDsurface*surface_background_std)
    surface_cell = max_response - surface_threshold
    surface_cell = max_response/np.std(max_response)

    return surface_cell

### Combine Three Different Segmentation Methods

In [8]:
def combine_images(inside_image, normalized_cell, surface_cell):
    level = 0.999
    combined_image = np.maximum(np.maximum(inside_image, normalized_cell), surface_cell);
    combined_image[combined_image<0] = 0
    combined_image = combined_image>level
    combined_image = binary_fill_holes(combined_image)

    # Label Connected Components
    labeled_image = label(combined_image)
    label_properties = regionprops(labeled_image)

    # Find Biggest Connected Component
    label_areas = np.zeros(np.size(label_properties[:]))
    for a in range(int(np.size(label_properties[:]))):
        label_areas[a] = label_properties[a].area
    max_label = np.argmax(label_areas, axis=None)

    # Take only the largest connected component.
    final_image = np.zeros(np.shape(labeled_image))
    final_image[labeled_image==max_label+1] = 1
    return final_image


### Run the Code

In [9]:
scales = [1, 2, 4]
nSTDsurface = 2
insideGamma = 0.7
insideBlur = 1
insideDilateRadius = 3
insideErodeRadius = 4

image_directory = '/archive/MIL/morrison/20201105_mitochondria_quantification/ilastik'
image_name = 'ControlCell8_cyto.tif'
image_path = os.path.join(image_directory, image_name)
image_data = np.array(imread(image_path))

padded_image_data = add_median_border(image_data)
inside_image = make_inside_image(padded_image_data, insideGamma, insideBlur, insideDilateRadius, insideErodeRadius)
normalized_cell = make_normalized_image(padded_image_data)
surface_cell = multiscale_surface_filter_3D(padded_image_data, scales)
final_image = combine_images(inside_image, normalized_cell, surface_cell)


### View in napari as a segmented label

In [10]:
with napari.gui_qt():
    viewer = napari.Viewer()  
    viewer.add_image(padded_image_data, name='raw data')
    viewer.add_labels(final_image, name='combo')

### View in napari as a mesh

In [10]:
# Use marching cubes to obtain the surface mesh of these ellipsoids
vertices, faces, normals, values = marching_cubes(final_image, 0.5)
surface = (vertices, faces, values)

with napari.gui_qt():
    viewer = napari.Viewer()  
    viewer.add_surface(surface, name='surface')


### Laplacian of Gaussian Filtering

In [43]:
def filter_log(input, sigma):
    """Same Sigma Value Assumed for All 3 Dimensions
    Approach Described Here: https://homepages.inf.ed.ac.uk/rbf/HIPR2/log.htm
    """
    
    w = np.ceil(5*sigma)
    x = np.arange(-w, w, 1)
    LoG = np.zeros((x.size,x.size,x.size))

    for i in range(int(x.size)):  
        for j in range(int(x.size)):
            for k in range(int(x.size)):
                # Gaussian
                g1 = x[i]**2/(2*sigma**2)
                g2 = x[j]**2/(2*sigma**2)
                g3 = x[k]**2/(2*sigma**2)
                # Laplacian
                l1 = -1/(np.pi*sigma**4)
                l2 = 1-((x[i]**2+x[j]**2+x[k]**2)/(2*sigma**2))
                # Combined
                LoG[i,j,k] = -l1*l2*np.exp(-g1-g2-g3)

    LoG = LoG - np.mean(LoG)
    im_LoG_response = signal.fftconvolve(input, LoG, mode='same')
    return im_LoG_response

def spatial_log(input, sigma):
    """Adopted from Philippe Roudot's uTrack3D Package
    First sigma value is for X and Y, second sigma value is for Z
    """
    w = np.ceil(5*np.max(sigma))
    x = np.arange(-w, w, 1)

    gx = np.zeros(np.shape(x))
    gz = np.zeros(np.shape(x))
    
    if np.size(sigma)==1:
        sigma[0:1]=sigma

    gx = np.exp(-(x**2)/(2*sigma[0]**2))
    gz = np.exp(-(x**2)/(2*sigma[1]**2))

    fg = signal.fftconvolve(input, gz[:, None, None], mode='same')
    fg = signal.fftconvolve(fg, gx[None, :, None], mode='same')
    fg = signal.fftconvolve(fg, gx[None, None, :], mode='same')

    gx2 = (x**2)*gx
    gz2 = (x**2)*gz

    fgz2 = signal.fftconvolve(input, gz2[:, None, None], mode='same')
    fgz2 = signal.fftconvolve(fgz2, gx[None, :, None], mode='same')
    fgz2 = signal.fftconvolve(fgz2, gx[None, None, :], mode='same')

    fgy2 = signal.fftconvolve(input, gz[:, None, None], mode='same')
    fgy2 = signal.fftconvolve(fgy2, gx2[None, :, None], mode='same')
    fgy2 = signal.fftconvolve(fgy2, gx[None, None, :], mode='same')

    fgx2 = signal.fftconvolve(input, gz[:, None, None], mode='same')
    fgx2 = signal.fftconvolve(fgx2, gx[None, :, None], mode='same')
    fgx2 = signal.fftconvolve(fgx2, gx2[None, None, :], mode='same')

    res = (2/sigma[0]**2+1/sigma[1]**2)*fg - ((fgx2+fgy2)/sigma[0]**4 + fgz2/sigma[1]**4);
    return res

### Find Maximum Filter Responses in 3D

In [40]:
def locmax3d(input, wdims):
    """ Adopted from Francois Aguet's Software
    https://github.com/francois-a/llsmtools/blob/master/psdetect3d/locmax3d.m
    [lm] = locmax3d(img, wdims, varargin) searches for local maxima in 3D 
    wdims = default values are 1 (Philippe's Pole detector) and 3 (Deepak's Blob Detector)
    """
    input=np.array(input)
    if np.ndim(wdims)==1:
        wx = wdims
        wy = wdims
        wz = wdims
        if np.mod(wx,2)==0 and np.mod(wy,2)==0 and np.mod(wz,0)==0:
           raise ValueError("Mask dimensions must be odd integers")
        
    if np.ndim(wdims)==3:
        wx = wdims[2]
        wy = wdims[1]
        wz = wdims[0]
        if np.mod(wx,2)==0 and np.mod(wy,2)==0 and np.mod(wz,0)==0:
           raise ValueError("Mask dimensions must be odd integers")
    
    if np.ndim(input) != 3:
        raise ValueError("Expected 3D Image")
        
    (nz, ny, nx) = input.shape
    lm2D = np.zeros((nz,ny,nx))
    for z in range(int(nz)):
        lm2D[z,:,:] = ndimage.maximum_filter(input[z,:,:], size=wx[0]*wy[0])

    lm = np.zeros((nz,ny,nx))
    b = (wz[0]-1)/2
    for z in range(int(nz)):
        start_idx = np.int(np.max([0, z-b]))
        stop_idx = np.int(np.min([nz,z+b]))
        temp_matrix = lm2D[start_idx:stop_idx,:,:]
        lm[z,:,:] = np.amax(temp_matrix, axis=0)

    lm[lm != input] = 0
    
    # Clear the Borders
    b = np.int((wz[0]+2))
    lm[0:b,:,:] = 0
    stop_idx = nz
    lm[stop_idx-b:stop_idx,:,:] = 0
    
    b = np.int((wy[0]+2))
    lm[:,0:b,:] = 0
    stop_idx = ny
    lm[:,stop_idx-b:stop_idx,:] = 0
    
    b = np.int((wx[0]+2))
    lm[:,:,0:b] = 0
    stop_idx = nx
    lm[:,:,stop_idx-b:stop_idx] = 0
    

    return lm

In [68]:
def filter_multiscale_LoG_ND(input,scales):
    #sigma = [1,2,5]
    #input = image_data
    
    (nz, ny, nx) = input.shape
    im_curl_LoG_response = np.zeros((nz, ny, nx))
    im_multiscale_LoG_response = np.zeros((nz, ny, nx))
    im_better_mask = np.zeros((nz, ny, nx))
    pixel_scale_map = np.zeros((nz, ny, nx))

    for scale_idx in range(int(np.size(sigma))):
        print("Processing Scale " + np.str(sigma[scale_idx]))
        im_curl_LoG_response = filter_log(input, sigma[scale_idx])
        
        if scale_idx == 1:
            im_multiscale_LoG_response = im_curl_LoG_response
            pixel_scale_map = np.ones((nz, ny, nx))
        
        else:
            im_better_mask = im_curl_LoG_response < im_multiscale_LoG_response
            im_multiscale_LoG_response[im_better_mask] = im_curl_LoG_response[im_better_mask]
            pixel_scale_map[im_better_mask] = scale_idx
    
    return im_multiscale_LoG_response, pixel_scale_map
                                          


In [69]:
image_directory = '/archive/MIL/morrison/20201105_mitochondria_quantification/ilastik'
image_name = 'ControlCell8_mito.tif'
image_path = os.path.join(image_directory, image_name)
image_data = np.array(imread(image_path))

#im_log = spatial_log(image_data, [sigma, sigma])
sigma = [1,2,5]
im_log, scales = filter_multiscale_LoG_ND(image_data,sigma)
lm = locmax3d(im_log, [3])

# im_LoG_response=img_as_float(im_LoG_response)
#image_max = ndimage.maximum_filter(im_LoG_response, size=sigma, mode='constant')
#coordinates = peak_local_max(im_LoG_response, min_distance=2*sigma)

Processing Scale 1
Processing Scale 2
Processing Scale 5


In [71]:
with napari.gui_qt():
    viewer = napari.Viewer()  
    #viewer.add_image(image_data, name='original data')
    #viewer.add_image(im_log, name='log filter')
    #viewer.add_image(lm, name='local max 2')
    viewer.add_image(scales)

    

In [177]:
test=[1,2,3]
print(test[-1])

3
